# Install Required Libraries

In [6]:
pip install librosa pandas scikit-learn joblib streamlit

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Define Emotion Label Extractor

In [10]:
def parse_emotion(filename):
    emotion_code = int(filename.split('-')[2])
    mapping = {
        1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad',
        5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprised'
    }
    return mapping.get(emotion_code)

# Define Audio Loading and MFCC Extraction

# Load Audio File

In [12]:
def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=22050)  # 22050 Hz sampling rate
    return signal, sr

# Extract MFCC Features

In [14]:
def extract_mfcc(signal, sr, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

# Build Feature Dataset

In [16]:
dataset_path = "C:\\Users\\akhil\\Desktop\\Emotion_detection_voice\\RAVDESS" # RAVDESS path
features = []

for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            signal, sr = load_audio(file_path)
            mfcc = extract_mfcc(signal, sr)
            emotion = parse_emotion(file)
            features.append([*mfcc, emotion])

# Create DataFrame
columns = [f'mfcc_{i+1}' for i in range(13)] + ['label']
df = pd.DataFrame(features, columns=columns)

# Optional: Save dataset
df.to_csv('emotion_features.csv', index=False)


# Train & Test split

In [18]:
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Train a Simple SVM Classifier


In [20]:
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

# Evaluate the Model

In [22]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.4652777777777778
              precision    recall  f1-score   support

       angry       0.65      0.80      0.72        76
        calm       0.39      0.61      0.48        77
     disgust       0.46      0.40      0.43        77
     fearful       0.57      0.57      0.57        77
       happy       0.45      0.31      0.37        77
     neutral       0.38      0.08      0.13        38
         sad       0.28      0.32      0.30        77
   surprised       0.49      0.43      0.46        77

    accuracy                           0.47       576
   macro avg       0.46      0.44      0.43       576
weighted avg       0.46      0.47      0.45       576



# Save the Trained Model

In [24]:
joblib.dump(model, 'emotion_model.joblib')

['emotion_model.joblib']

In [26]:
loaded_model = joblib.load('emotion_model.joblib')